In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/22_STTR_phase1.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [23]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "", "Is Principal Investigator US Citizen or Permanent Resident":"",
           "Principal Investigator_Phone": "", "Principal Investigator_Company": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          "Research Institution Phone":"", "TAV Subtopics":"", "Duration":""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    ### basic info
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Subtopic Title'] = filterHTMLstr(getResearchSubTopicTitle(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    dic['Phase 1 Contract #'] = filterHTMLstr(getPhase1Number(bs))
    
    divs1 = bs.find_all("div")   #info
    Name=[]
    Email=[]
    Address=[]
    Phone=[]

    for i in range(len(divs1)):
        str = divs1[i].string
        if (str is not None):
            if (str.find("Firm") != -1):
                # Firm
                dic["Small Business Concern_Firm"]=filterHTMLstr(divs1[i+2].strong.get_text())
            elif (str.find("Name") != -1):
                # Name
                Name.append(i + 2)
            elif (str.find("E-mail") != -1):
                # Email
                Email.append(i + 2)
            elif (str.find("Address") != -1):
                # Address
                Address.append(i + 2)
            elif (str.find("Phone") != -1):
                # Phone
                Phone.append(i + 2)

    dic["Research Institution Name"]=filterHTMLstr(divs1[Name[0]].strong.get_text())
    dic["Principal Investigator_Name"]=filterHTMLstr(divs1[Name[1]].strong.get_text())
    dic["Business Official_Name"]=filterHTMLstr(divs1[Name[2]].strong.get_text())
    
    dic["Principal Investigator_E-mail"]=filterHTMLstr(divs1[Email[0]].strong.get_text())
    dic["Business Official_E-mail"] = filterHTMLstr(divs1[Email[1]].strong.get_text())
    
    dic["Small Business Concern_Address"] = filterHTMLstr(divs1[Address[0]].strong.get_text())
    dic["Research Institution Address"] = filterHTMLstr(divs1[Address[1]].strong.get_text())
    dic["Principal Investigator_Address"] = filterHTMLstr(divs1[Address[2]].strong.get_text())
    dic["Business Official_Address"] = filterHTMLstr(divs1[Address[3]].strong.get_text())
    
    
    dic["Small Business Concern_Phone"] = filterHTMLstr(divs1[Phone[0]].strong.get_text())
    dic["Research Institution Phone"] = filterHTMLstr(divs1[Phone[1]].strong.get_text())
    dic["Principal Investigator_Phone"] = filterHTMLstr(divs1[Phone[2]].strong.get_text())
    dic["Business Official_Phone"] = filterHTMLstr(divs1[Phone[3]].strong.get_text())

    divs2=bs.find_all("strong")
    for i in range(len(divs2)):
        str=divs2[i].get_text()
        #print(str)
        if(str is not None):
            if(str.find("Begin")!=-1):
                TRL_BE=str.replace('\n','').split()
            elif (divs2[i].parent.get_text().find("Duration") != -1):
                dic["Duration"] = filterHTMLstr(str)

    TRL_Begin=TRL_BE[1]
    TRL_End=TRL_BE[3]
    dic["TRL_Begin"]=TRL_Begin
    dic["TRL_End"] = TRL_End

    Abstract=[]
    NASA_applicants=[]
    Non_NASA_applicants=[]
    paras=bs.find_all("p")   #  Abstract, NASA applicants, Non-NASA applicants
    for i in range(len(paras)):
        #print(paras[i].string)
        if(paras[i] is not None):
            #print(paras[i].parent.get_text())
            para = paras[i].get_text()
            para = filterHTMLstr(para)
            para_parent_str=paras[i].parent.get_text()
            if(len(para)>0):
                if (para_parent_str.find("Technical Abstract") != -1):
                    Abstract.append(para)
                elif(para_parent_str.find("Potential NASA Applications") != -1):
                    NASA_applicants.append(para)
                elif(para_parent_str.find("Potential Non-NASA Applications") != -1):
                    Non_NASA_applicants.append(para)

    dic["Technical Abstract"]=" ".join(Abstract)
    dic["Potential NASA applications"]=" ".join(NASA_applicants)
    dic["Potential non-NASA applications"]=" ".join(Non_NASA_applicants)
    return dic

In [26]:
Directory_path = "../Datasets/22/sttr/phase1"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  53
../Datasets/22/sttr/phase1/STTR-22-1-T5.05-1029.html
../Datasets/22/sttr/phase1/STTR-22-1-T4.01-1810.html
../Datasets/22/sttr/phase1/STTR-22-1-T8.07-1524.html
../Datasets/22/sttr/phase1/STTR-22-1-T10.03-2033.html
../Datasets/22/sttr/phase1/STTR-22-1-T12.07-1654.html
../Datasets/22/sttr/phase1/STTR-22-1-T10.04-1383.html
../Datasets/22/sttr/phase1/STTR-22-1-T7.04-1851.html
../Datasets/22/sttr/phase1/STTR-22-1-T13.01-2449.html
../Datasets/22/sttr/phase1/STTR-22-1-T10.04-1854.html
../Datasets/22/sttr/phase1/STTR-22-1-T12.07-1475.html
../Datasets/22/sttr/phase1/STTR-22-1-T5.05-2038.html
../Datasets/22/sttr/phase1/STTR-22-1-T10.05-2150.html
../Datasets/22/sttr/phase1/STTR-22-1-T5.05-2204.html
../Datasets/22/sttr/phase1/STTR-22-1-T10.03-1993.html
../Datasets/22/sttr/phase1/STTR-22-1-T7.04-2186.html
../Datasets/22/sttr/phase1/STTR-22-1-T11.06-2113.html
../Datasets/22/sttr/phase1/STTR-22-1-T5.04-1819.html
../Datasets/22/sttr/phase1/STTR-22-1-T7.04-1066.html
../Da

# Test

In [20]:
path = "../Datasets/22/sttr/phase1/STTR-22-1-T5.05-1029.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [28]:
### Technical Abstract and Potential Commercial Application and NASA Application
def getTechAbstract(bs):
    Abstract=[]
    NASA_applicants=[]
    Non_NASA_applicants=[]
    paras=bs.find_all("p")   #  Abstract, NASA applicants, Non-NASA applicants
    for i in range(len(paras)):
        #print(paras[i].string)
        if(paras[i] is not None):
            #print(paras[i].parent.get_text())
            para = paras[i].get_text()
            para = filterHTMLstr(para)
            para_parent_str=paras[i].parent.get_text()
            if(len(para)>0):
                if (para_parent_str.find("Technical Abstract") != -1):
                    Abstract.append(para)
                elif(para_parent_str.find("Potential NASA Applications") != -1):
                    NASA_applicants.append(para)
                elif(para_parent_str.find("Potential Non-NASA Applications") != -1):
                    Non_NASA_applicants.append(para)
    return " ".join(Abstract)

def getNonNASAApplication(bs):
    Abstract=[]
    NASA_applicants=[]
    Non_NASA_applicants=[]
    paras=bs.find_all("p")   #  Abstract, NASA applicants, Non-NASA applicants
    for i in range(len(paras)):
        #print(paras[i].string)
        if(paras[i] is not None):
            #print(paras[i].parent.get_text())
            para = paras[i].get_text()
            para = filterHTMLstr(para)
            para_parent_str=paras[i].parent.get_text()
            if(len(para)>0):
                if (para_parent_str.find("Technical Abstract") != -1):
                    Abstract.append(para)
                elif(para_parent_str.find("Potential NASA Applications") != -1):
                    NASA_applicants.append(para)
                elif(para_parent_str.find("Potential Non-NASA Applications") != -1):
                    Non_NASA_applicants.append(para)
    return " ".join(NASA_applicants)

def getNASAApplication(bs):
    Abstract=[]
    NASA_applicants=[]
    Non_NASA_applicants=[]
    paras=bs.find_all("p")   #  Abstract, NASA applicants, Non-NASA applicants
    for i in range(len(paras)):
        #print(paras[i].string)
        if(paras[i] is not None):
            #print(paras[i].parent.get_text())
            para = paras[i].get_text()
            para = filterHTMLstr(para)
            para_parent_str=paras[i].parent.get_text()
            if(len(para)>0):
                if (para_parent_str.find("Technical Abstract") != -1):
                    Abstract.append(para)
                elif(para_parent_str.find("Potential NASA Applications") != -1):
                    NASA_applicants.append(para)
                elif(para_parent_str.find("Potential Non-NASA Applications") != -1):
                    Non_NASA_applicants.append(para)
    return " ".join(Non_NASA_applicants)

### TRL
def getTRL(bs):
    TRL = ""
    TRLStart = ""
    TRLEnd = ""
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Estimated Technology Readiness Level (TRL) : " in s)
    pre_pos = bs.find("span", string=[is_the_only_string_within_a_tag])
    TRL = pre_pos.parent.get_text()
    regDoubleNum = re.compile(r'\d+\d+')
    regSingleNum = re.compile(r'\d+')
    nums = re.findall(regDoubleNum, TRL) if len(re.findall(regDoubleNum, TRL)) > 0 else re.findall(regSingleNum, TRL)
    TRLStart = nums[0] if len(nums) > 0 else ""
    TRLEnd = nums[1] if len(nums) == 2 else TRLStart
    return [TRLStart, TRLEnd]

### TAV
# def getTAV(bs):
#     Ps = bs.find_all("p")
#     res = ""
#     for p in Ps:
#         if "Technology Available (TAV) Subtopics" in p.get_text():
#             info = p
#             infos = [BeautifulSoup(_,'html.parser').text.strip() for _ in str(info).split('<br/>')]
#             for i in infos[1:]:
#                 res += filterHTMLstr(i)
#     return res

In [18]:
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and "Proposal Number:" in s)

pre_pos = bs.find("div", string=[is_the_only_string_within_a_tag])
# pre_pos
proposalNumber = filterHTMLstr(pre_pos.find_next_sibling().get_text())
print(proposalNumber)

19-2- T6.06-2448


In [22]:
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and (("Proposal Number:" in s) or ("PROPOSAL NUMBER:" in s)))

pre_pos = bs.find("div", string=[is_the_only_string_within_a_tag])
proposalNumber = filterHTMLstr(pre_pos.find_next_sibling().get_text())
proposalNumber

'22-1- T5.05-1029'

In [25]:
### Proposal Number
def getProposalNumber(bs):
    divs2=bs.find_all("strong")
    for i in range(len(divs2)):
        str=divs2[i].get_text()
        #print(str)
        if(str is not None):
            if(str.find("Begin")!=-1):
                TRL_BE=str.replace('\n','').split()
            elif (divs2[i].parent.get_text().find("Proposal Number") != -1):
                # Proposal number
                str = filterHTMLstr(str)
                str = str.replace("21-1-", "21-1")
                dic["Proposal Number"] = filterHTMLstr(str)
            elif (divs2[i].parent.get_text().find("Phase 1 Contract #") != -1):
                # Phase 1 Contract Number
                dic["Phase 1 Contract #"] = filterHTMLstr(str)
            elif (divs2[i].parent.get_text().find("Subtopic Title") != -1):
                # Subtopic Title
                dic["Subtopic Title"] = filterHTMLstr(str)
            elif (divs2[i].parent.get_text().find("Proposal Title") != -1):
                # Proposal Title
                dic["Proposal Title"] = filterHTMLstr(str)
    return proposalNumber

### Phase 1 Contract Number
def getPhase1Number(bs):
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and (("PHASE-I CONTRACT NUMBER" in s) or ("PHASE-1 CONTRACT NUMBER" in s) or ("PHASE 1 CONTRACT NUMBER" in s) or ("Phase 1 Contract #" in s)))
    Phase1Number = ""
    pre_pos = bs.find("div", string=[is_the_only_string_within_a_tag])
    if pre_pos != None:
        Phase1Number = filterHTMLstr(pre_pos.find_next_sibling().get_text())
    return Phase1Number

### Research Topic and project title
def getResearchSubTopicTitle(bs):
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s.parent.string in s and (("Subtopic Title" in s) or ("SUBTOPIC TITLE:" in s)))
    ResearchSubTopicTitle = ""
    pre_pos = bs.find("div", string=[is_the_only_string_within_a_tag])
    ResearchSubTopicTitle = filterHTMLstr(pre_pos.find_next_sibling().get_text())
    return ResearchSubTopicTitle

def getProjectTitle(bs):
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and (("Proposal Title" in s) or ("PROPOSAL TITLE:" in s)))
    ProjectTitle = ""
    pre_pos = bs.find("div", string=[is_the_only_string_within_a_tag])
    ProjectTitle = filterHTMLstr(pre_pos.find_next_sibling().get_text())
    return ProjectTitle

In [8]:
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and "(SBC)" in s)
pre_pos = bs.find("b", string=[is_the_only_string_within_a_tag])
a = pre_pos.find_next_siblings("br")
print(a[0].next_sibling.get_text())

AttributeError: 'NoneType' object has no attribute 'find_next_siblings'

In [28]:
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and "(SBC)" in s)
pre_pos = bs.find("b", string=[is_the_only_string_within_a_tag])
pre_pos.parent

<div class="divTableCell">
<b><u>SMALL BUSINESS CONCERN (SBC):</u></b>
<br/>
                    H2O Insights, LLC<br/>
                    12794 East Sahuaro Drive<br/>
                    Scottsdale AZ  85259 - 4331<br/>
                    Phone: (480) 209-9140
                </div>

In [6]:
### Small Business and Investigator Info
def getSMBAndRIInfo(bs):
    smbName = ""
    smbAddress = ""
    smbPhone = ""
    riName = ""
    riAddress = ""
    riPhone = ""
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "(SBC):" in s)
    pre_pos = bs.find("b", string=[is_the_only_string_within_a_tag])
    info = pre_pos.parent
    infos = [BeautifulSoup(_,'html.parser').text.strip() for _ in str(info).split('<br/>')]
    smbName = infos[1].strip() if len(infos) > 1 else ""
    address1 = infos[2].strip() if len(infos) > 2 else ""
    replace = infos[3].replace('\n', ',').strip() if len(infos) > 3 else ""
    smbPhone = infos[4].strip().replace("Phone:", "").lstrip() if len(infos) > 4 else ""
    smbAddress = address1 + ", " + replace if len(address1.strip()) > 0 else ""
    
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "(RI):" in s)
    pre_pos = bs.find("b", string=[is_the_only_string_within_a_tag])
    info = pre_pos.parent
    infos = [BeautifulSoup(_,'html.parser').text.strip() for _ in str(info).split('<br/>')]
    riName = infos[1].strip() if len(infos) > 1 else ""
    address1 = infos[2].strip() if len(infos) > 2 else ""
    replace = infos[3].replace('\n', ',').strip() if len(infos) > 3 else ""
    riPhone = infos[4].strip().replace("Phone:", "").lstrip() if len(infos) > 4 else ""
    riAddress = address1 + ", " + replace if len(address1.strip()) > 0 else ""
    
    return [smbName, smbAddress, smbPhone, riName, riAddress, riPhone]


In [32]:
duration = ""
regDuration = re.compile(r'\d+')
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and "Duration:" in s)
st = bs.find("strong", string=[is_the_only_string_within_a_tag])
if st != None:
    # print(st.parent)
    duration = re.findall(regDuration, st.parent.get_text())[0] if len(re.findall(regDuration, st.parent.get_text())) > 0 else ""
duration

''

In [7]:
def getDuration(bs):
    duration = ""
    regDuration = re.compile(r'\d+')
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Duration:" in s)
    st = bs.find("strong", string=[is_the_only_string_within_a_tag])
    if st != None:
        # print(st.parent)
        duration = re.findall(regDuration, st.parent.get_text())[0] if len(re.findall(regDuration, st.parent.get_text())) > 0 else ""
    return duration

In [92]:
def is_the_only_string_within_a_tag(s):
    """Return True if this string is the only child of its parent tag."""
    return (s == s.parent.string and "Name:" in s)
bs.find_all("div", string=[is_the_only_string_within_a_tag])[0].find_next_sibling("div").get_text()

'\n            \n        Christine Escobar\n        \n'

In [8]:
def getInvestigatorInfo(bs):
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Name:" in s)
    name = bs.find_all("div", string=[is_the_only_string_within_a_tag])[0].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "E-mail:" in s)
    email = bs.find_all("div", string=[is_the_only_string_within_a_tag])[0].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Address:" in s)
    address = bs.find_all("div", string=[is_the_only_string_within_a_tag])[0].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Phone:" in s)
    phone = bs.find_all("div", string=[is_the_only_string_within_a_tag])[0].find_next_sibling("div").get_text()
    return [name, email, address, phone]

In [9]:
def getBusinessOfficialInfo(bs):
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Name:" in s)
    name = bs.find_all("div", string=[is_the_only_string_within_a_tag])[1].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "E-mail:" in s)
    email = bs.find_all("div", string=[is_the_only_string_within_a_tag])[1].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Address:" in s)
    address = bs.find_all("div", string=[is_the_only_string_within_a_tag])[1].find_next_sibling("div").get_text()
    def is_the_only_string_within_a_tag(s):
        """Return True if this string is the only child of its parent tag."""
        return (s == s.parent.string and "Phone:" in s)
    phone = bs.find_all("div", string=[is_the_only_string_within_a_tag])[1].find_next_sibling("div").get_text()
    return [name, email, address, phone]

In [9]:
# def getTechnologyTaxonomyMapping(bs):
#     divs = bs.find_all("div")
#     technologyTaxMapping = ""
#     for i in range(len(divs)):
#         div = divs[i]
#         if "TECHNOLOGY TAXONOMY MAPPING" in div.get_text():
#             technologyTaxMapping = divs[i + 1].get_text().strip().replace('\n', ';')
#     return technologyTaxMapping